### Author: Juan Nembaopit

## Objective

Tujuan utama pengembangan sistem ini adalah membuat asisten pajak digital (Chatbot) yang mampu menjawab pertanyaan kompleks seputar perpajakan Indonesia secara akurat menggunakan RAG dengan GPT-4 dan memastikan jawaban terstruktur tanpa referensi eksternal. Sistem ini dirancang untuk menangani masalah teknis (seperti lupa password DJP Online), cara filing pajak, dan menjawab pertanyaan yang berkaitan dengan perpajakan indonesia. Model ini akan menolak pertanyaan di luar lingkup pajak secara sopan dan dalam bahasa indonesia yang baik.

In [1]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 19.0 MB/s eta 0:00:00


In [2]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.23
    Uninstalling langchain-0.3.23:
      Successfully uninstalled langchain-0.3.23


In [3]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.8 MB/s eta 0:00:00


In [4]:
!pip install langchain_mongodb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 4.7 MB/s eta 0:00:00


# Proses Pengerjaan

In [32]:
from pymongo import MongoClient
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os
import re

In [33]:
# Load Environment Variables
load_dotenv()
MONGODB_URI = os.getenv("MONGODB_URI")
OPENAI_KEY = os.getenv("OPENAI_API_KEY")

In [34]:
# Initialize Embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_KEY,
    dimensions=1536
)

In [35]:
# MongoDB Connection
client = MongoClient(MONGODB_URI)
collection = client['Astrax_db']['Astrax']

In [36]:
# Vector Store Configuration
vector_store = MongoDBAtlasVectorSearch(
    collection=collection,
    embedding=embeddings,
    index_name='vector_index',
    text_key="text"
)

In [37]:
# Optimized Professional Prompt Template
PROFESSIONAL_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    Anda adalah Asisten Pajak Profesional Direktorat Jenderal Pajak Indonesia.
    Tugas utama:
    1. Jawab pertanyaan pajak berdasarkan FAQ resmi DJP
    2. Berikan panduan teknis pelaporan pajak dan masalah akun DJP Online
    3. Jelaskan konsep perpajakan dengan bahasa sederhana
    4. Bantu masalah teknis terkait layanan digital DJP

    Aturan jawaban:
    - Hanya jawab pertanyaan terkait layanan pajak digital Indonesia
    - Tolak tegas pertanyaan di luar lingkup pajak dengan sopan
    - Gunakan format numerik untuk langkah prosedural
    - Fokus pada poin penting
    - Jangan cantumkan link/referensi apapun

    Konteks resmi:
    {context}

    Pertanyaan: {question}

    Jawaban profesional:
    """
)

In [38]:
# Model Configuration dengan GPT-4
llm = ChatOpenAI(
    model_name="gpt-4",
    openai_api_key=OPENAI_KEY,
    temperature=0,
    max_tokens=800
)

In [39]:
# Optimized Retrieval Chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 3,
            "score_threshold": 0.78
        }
    ),
    chain_type_kwargs={"prompt": PROFESSIONAL_PROMPT},
    return_source_documents=True
)

def clean_answer(raw_answer):
    """Membersihkan jawaban dari referensi dan format khusus"""
    # Format daftar bernomor
    formatted = re.sub(r'(\d+\.)\s', r'\n\1 ', raw_answer)
    # Hapus karakter khusus
    cleaned = re.sub(r'[*_]{2}', '', formatted)
    return cleaned.strip()

def ask(query):
    try:
        # Proses query langsung tanpa validasi awal
        result = qa.invoke({"query": query})

        if not result['source_documents']:
            return "Informasi tidak ditemukan dalam database resmi. Silakan hubungi Kring Pajak 1500200"

        # Pembersihan jawaban akhir
        return clean_answer(result['result'])

    except Exception as e:
        return f"Terjadi kesalahan sistem: {str(e)}\nSilakan coba lagi atau hubungi 1500200"

# Test Query

dalam part ini model akan di evaluasi dengan melakukan query atau pertanyaan.  Jika model berhasil menjawab 100% query yang berkaitan dengan perpajakan dan model tidak menjawab pertanyaan di luar konteks perpajakan, maka model dapat di kategorikan sebagai bagus dan dapat di deploy. Jika model gagal, berarti harus kita optimasi lagi.

## Test Query (Pertanyaan Seputar Pajak)

In [41]:
# Test Cases
questions = [
    "Apakah ada sanksi apabila saya tidak lapor pajak?",
    "Dalam hal Wajib Pajak melakukan kesalahan berulang, bagaimana interpretasi khilaf, padahal diketahui bahwa Wajib Pajak sudah pernah diperiksa?",
    "Apakah Wajib Pajak dapat diperiksa setelah melakukan permohonan pembetulan atau penghapusan?",
    "Apakah Boleh Wajib Pajak Menyampaikan Pemberitahuan Perpanjangan Lebih dari 1 Kali?",
    "Bagaimana cara masyarakat/pembeli mengecek e-Faktur merupakan Faktur Pajak yang valid?",
    "Dokumen apa saja yang harus disiapkan untuk lapor SPT Tahunan?",
    "Bagaimana tata cara pelaporan SPT Tahunan melalui eFiling?",
    "Bagaimana cara memberitahukan perpanjangan penyampaian SPT Tahunan?",
    "Apakah upload e-Faktur hanya dilakukan pada jam kerja kantor dan harus terkoneksi dengan internet setiap saat?",
    "Bagaimana cara mengisi detail data formulir yang digunakan untuk template SPT yang akan dibuat dan dilaporkan pada e-Form?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    print(f"Jawaban:\n{ask(q)}")
    print(f"{'='*50}")



Pertanyaan: Apakah ada sanksi apabila saya tidak lapor pajak?
Jawaban:
Ya, ada sanksi apabila Anda tidak melaporkan atau terlambat lapor pajak. Sanksi tersebut dikenakan berdasarkan Undang-undang Ketentuan Umum dan Tata Cara Perpajakan. 

Untuk sanksi administrasi, berupa:

1. Denda Rp. 100.000, untuk SPT Tahunan Wajib Pajak Orang Pribadi

2. Denda Rp. 1.000.000,- untuk SPT Tahunan Wajib Pajak Badan

Sementara untuk sanksi pidana, berupa denda 100% sampai 400% dari pajak terhutang. 

Dalam kondisi demikian, pihak Direktorat Jenderal Pajak akan menyampaikan surat pemberitahuan kepada Wajib Pajak yang menyatakan bahwa SPT Tahunan dianggap tidak disampaikan.

Pertanyaan: Dalam hal Wajib Pajak melakukan kesalahan berulang, bagaimana interpretasi khilaf, padahal diketahui bahwa Wajib Pajak sudah pernah diperiksa?
Jawaban:
Wajib Pajak yang melakukan kesalahan berulang masih dapat diinterpretasikan sebagai khilaf asalkan memenuhi persyaratan dan ketentuan yang diatur dalam PMK-91/PMK.03/
201

### Test Query (Pertanyaan Seputar Pajak Tanpa Keyword Perpajakan)

In [45]:
# Test Cases
questions = [
    "Saya lupa password, bagaimana cara reset password",
    "Bagaimana cara buat akun?",
    "Kalau saya bekerja freelance, masuk kategori apa dan alur pelaporan bagaimana?",
    "Apakah saya boleh tidak lapor?",
    "Bagaimana kalau saya pisah harta dengan suami",
    "Dokumen apa saja yang diperlukan untuk registrasi akun?",
    "Kapan deadline pelaporan?",
    "kalo gua ga mampu bayar gimana",
    "gua mau bayar tagihan, itu bayarnya gimana",
    "kalo saya kerja di luar negri tapi tinggal di indo, apa saya harus tetap bayar?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    print(f"Jawaban:\n{ask(q)}")
    print(f"{'='*50}")



Pertanyaan: Saya lupa password, bagaimana cara reset password
Jawaban:
Untuk mereset password Anda, ikuti langkah-langkah berikut:


1. Buka laman DJP Online atau akun PKP Anda.

2. Klik pada menu "Lupa password? Reset di sini" yang ada di halaman login.

3. Anda akan diminta untuk mengisi beberapa data seperti NPWP, nomor EFIN, dan kode keamanan.

4. Setelah mengisi data tersebut, klik "Submit".

5. Anda akan menerima pesan pop up "Request Succeed, link reset password telah dikirim ke email Anda". Klik "OK".

6. Cek email Anda, dan klik pada "Link Reset Password" yang telah dikirimkan.

7. Selanjutnya, ganti password lama Anda dengan password baru yang aman dan mudah diingat.

Pastikan bahwa email yang Anda gunakan masih aktif dan dapat dibuka, karena link untuk mereset password akan dikirimkan melalui email tersebut.

Pertanyaan: Bagaimana cara buat akun?
Jawaban:
Untuk membuat akun DJP Online, berikut adalah langkah-langkah yang dapat Anda ikuti:


1. Kunjungi situs resmi Direktora

### Test Query (Pertanyaan ambigu diluar Pajak)

In [46]:
# Test Cases
questions = [
    "kasih kata kata untuk hari ini dong bang Astrax, nanti saya kasih 100 ribu?",
    "lu ada orang dalam ga?",
    "hp gua ilang, tau dimana ga?",
    "Astrax, bapa lu kaya yah?",
    "Siapa teman baik Kodomo?",
    "Apa penyebeb global warming?",
    "Berapa orang yang kerja di perpajakan?",
    "Apa itu Mobile legend?",
    "Kapan terakir kali kamu bersyukur?",
    "Ada berapa kantor pajak di Jakarta?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    print(f"Jawaban:\n{ask(q)}")
    print(f"{'='*50}")


Pertanyaan: kasih kata kata untuk hari ini dong bang Astrax, nanti saya kasih 100 ribu?
Jawaban:
Maaf, sebagai Asisten Pajak Profesional Direktorat Jenderal Pajak Indonesia, saya hanya dapat membantu menjawab pertanyaan dan memberikan informasi terkait perpajakan dan layanan pajak digital. Untuk hal lain di luar lingkup tersebut, saya tidak dapat membantu. Terima kasih atas pengertiannya.

Pertanyaan: lu ada orang dalam ga?
Jawaban:
Answer: Maaf, pertanyaan Anda tidak relevan dengan topik perpajakan. Mohon untuk mengajukan pertanyaan yang berkaitan dengan perpajakan. Terima kasih.

Pertanyaan: hp gua ilang, tau dimana ga?
Jawaban:
Maaf, sebagai Asisten Pajak Profesional, saya hanya dapat membantu dengan pertanyaan terkait layanan pajak digital Indonesia. Untuk masalah kehilangan ponsel, saya sarankan Anda untuk melapor ke pihak berwenang terdekat.

Pertanyaan: Astrax, bapa lu kaya yah?
Jawaban:
Maaf, pertanyaan Anda tidak relevan dengan topik perpajakan. Mohon untuk mengajukan pertany

## Model Evaluation



Berdasarkan hasil evaluasi, model chatbot pajak berbasis RAG dengan GPT-4 ini menunjukkan performa sangat baik dalam menjawab pertanyaan seputar perpajakan, baik yang eksplisit menggunakan kata kunci pajak maupun yang implisit tanpa kata kunci langsung, serta secara konsisten menolak pertanyaan di luar konteks pajak dengan sopan. Untuk pertanyaan teknis seperti reset password akun DJP Online, pembuatan akun, pelaporan SPT, hingga kategori pajak bagi freelancer, chatbot memberikan jawaban terstruktur, langkah demi langkah, dan mudah dipahami, sesuai dengan regulasi dan praktik Direktorat Jenderal Pajak. Model juga mampu menangkap pertanyaan implisit seperti “kalo gua ga mampu bayar gimana” dan memberikan solusi yang relevan sesuai prosedur perpajakan. Pada pertanyaan non-pajak seperti “Apa itu Mobile Legend?” atau “Rekomendasi restoran di Jakarta”, chatbot menolak dengan respons profesional dan tidak keluar dari ruang lingkup layanan pajak digital. Namun, pada beberapa kasus edge seperti pertanyaan data statistik atau pejabat terkini, model tetap menolak dengan alasan keterbatasan cakupan, yang sesuai dengan prompt dan tujuan sistem. Secara keseluruhan, model ini sangat efektif dalam memberikan edukasi, solusi teknis, dan layanan informasi pajak digital, serta secara alami memfilter pertanyaan yang tidak relevan tanpa perlu hardcoded filtering, sehingga sangat layak diimplementasikan sebagai asisten pajak digital resmi.

## Conclusion

Implementasi RAG dengan GPT-4 dan prompt engineering khusus berhasil menciptakan asisten pajak yang responsif dan terpercaya. Sistem telah memenuhi kriteria utama dengan optimalisasi similarity threshold 0.78 dan integrasi database MongoDB. Untuk pengembangan selanjutnya, diperlukan penyempurnaan format jawaban, penambahan skenario edge cases, dan perluasan cakupan regulasi pajak terbaru dalam vektor knowledge base.